In [1]:
import pandas as pd
df = pd.read_csv("./yBTC-truncated.csv")
df['unix'] = pd.to_datetime(df['unix'],unit='s')
df.set_index('unix')

,open,high,low,close
unix,,,,
2020-10-14 22:00:00,11382.32,11420.00,11382.32,11401.81
2020-10-14 23:00:00,11400.50,11443.26,11388.52,11428.84
2020-10-15 00:00:00,11430.49,11431.59,11396.79,11399.17
2020-10-15 01:00:00,11402.78,11424.67,11374.88,11398.02
2020-10-15 02:00:00,11393.47,11409.94,11367.07,11371.43
...,...,...,...,...
2022-01-22 06:00:00,35750.46,35987.85,35366.70,35706.19
2022-01-22 07:00:00,35735.99,35889.63,35111.00,35398.60
2022-01-22 08:00:00,35408.23,35825.00,35408.23,35613.89


In [2]:
# threshold = int(0.75 * len(df))

# df_train = df.iloc[:threshold,:]
# df_test = df.iloc[threshold:,:]

In [3]:
import pickle
def drop_columns(df, colnames=['volumefrom', 'volumeto']):
    for cn in colnames:
        try:
          df.drop(cn, inplace=True, axis=1)
        except:
            pass
    return df

def add_ema(df, ema=28):
    # df['backward_ewm'] = df['Close'].ewm(span=ema,min_periods=0,adjust=False,ignore_na=False).mean()
    df['ewm'] = df['close'].ewm(span=ema,min_periods=0,adjust=False,ignore_na=False).mean()
    return df

def add_macd(df, e1=12, e2=26, e3=9):
    exp1 = df['close'].ewm(span=e1, adjust=False).mean()
    exp2 = df['close'].ewm(span=e2, adjust=False).mean()
    macd = exp1 - exp2
    macd_signal = macd.ewm(span=e3, adjust=False).mean()
    df['macd_histo'] = macd - macd_signal
    return df

def add_returns(df):
    df['return'] = df['close'].pct_change(1)
    return df


def prepare(df):
    df = add_returns(add_macd(add_ema(drop_columns(df))))
    return df

In [4]:
# df_train_prepared = prepare(df_train).set_index('unix')
# df_test_prepared = prepare(df_test).set_index('unix')
df_trunc = prepare(df).set_index('unix')

In [5]:
# df_train_prepared.to_pickle('./train_prep.pkl')
# df_test_prepared.to_pickle('./test_prep.pkl')
df_trunc.to_pickle('./trunc_data.pkl')

In [6]:
df_trunc

,open,high,low,close,ewm,macd_histo,return
unix,,,,,,,
2020-10-14 22:00:00,11382.32,11420.00,11382.32,11401.81,11401.810000,0.000000,NaN
2020-10-14 23:00:00,11400.50,11443.26,11388.52,11428.84,11403.674138,1.724991,0.002371
2020-10-15 00:00:00,11430.49,11431.59,11396.79,11399.17,11403.363508,0.818354,-0.002596
2020-10-15 01:00:00,11402.78,11424.67,11374.88,11398.02,11402.994990,0.128005,-0.000101
2020-10-15 02:00:00,11393.47,11409.94,11367.07,11371.43,11400.818094,-2.014403,-0.002333
...,...,...,...,...,...,...,...
2022-01-22 06:00:00,35750.46,35987.85,35366.70,35706.19,37817.636679,-51.673015,-0.001129
2022-01-22 07:00:00,35735.99,35889.63,35111.00,35398.60,37650.806563,-57.828689,-0.008614
2022-01-22 08:00:00,35408.23,35825.00,35408.23,35613.89,37510.329559,-35.655211,0.006082
